# OpenPoseでkafkaの画像を処理する

kafkaの一連の画像ファイルを [OpenPose](https://github.com/CMU-Perceptual-Computing-Lab/openpose) で処理します。処理後の画像ファイルは kafka の別トピックに送ります。

# 事前チェック

このNotebookの処理は「04-001-GPUインスタンスを起動する.ipynb」で起動したGPUインスタンスを利用することを想定しています。

それ以外のノードに対して環境構築を行う場合は、以下の条件を満たすように準備を行ってください。

1. 対象となるノードを Ansible で操作できる
1. GPUが利用可能であること
1. [nvidia-docker](https://github.com/NVIDIA/nvidia-docker)のセットアップが済んでいること
1. 構築に利用するツールがインストールされていること
  - rsync
  - docker-compose

GPUインスタンスのVCノードを Ansible で操作できることを確認します。

In [ ]:
target = 'openpose-04'

In [ ]:
!ansible {target} -m ping

GPUが利用できることを確認します。`/dev/nvidia0` があることを確認します。

In [ ]:
!ansible {target} -a 'ls -la /dev/nvidia0'

Docker のランタイムとして nvidia が指定できることを確認します。

nvidia のランタイムが登録されている場合は、以下のような出力となります。
```
{nvidia-container-runtime []}
```

In [ ]:
!ansible {target} -a 'docker info -f "{{%raw%}}{{{{.Runtimes.nvidia}}}}{{%endraw%}}"'

必要なコマンドがインストールされていることを確認します。

In [ ]:
!ansible {target} -a 'which rsync'

In [ ]:
!ansible {target} -a 'docker-compose version'

# パラメータの指定

処理画像の取得元となる kafka のトピックを指定してください。

In [ ]:
topic_src = 'distributed-video1'

OpenPoseで処理した画像の送り先となる kafka のトピックを指定してください。

In [ ]:
topic_dst = 'distributed-video1-openpose'

Kafkaの brokerのアドレスを指定してください。

In [ ]:
kafka_brokers = [
    'broker-0:9092',
    'broker-1:9092',
    'broker-2:9092',   
]

broker の各ノードのホスト名とIPアドレスの対応を指定してください。

In [ ]:
kafka_hosts = {
    'broker-0': '172.30.2.10',
    'broker-1': '172.30.2.11',
    'broker-2': '172.30.2.12',
}

処理対象となる画像を取得する際のコンシューマグループIDを指定してください。

> コンシューマが、どこまでKafkaのトピックを取得したのかは Kafka で管理されています。ここで指定した値は、Kafkaがコンシューマを特定するための用いる値となります。

In [ ]:
consumer_group_id = 'openpose-001'

OpenPoseで処理を行う画像を間引く場合は次のセルのコメントを外して実行してください。

In [ ]:
# openpose_opts = ['-D', '3', ]     # この場合、画像3枚あたり1枚だけ OpenPoseで処理して、あとは捨てる

# OpenPoseの実行

## 設定ファイルの配置

In [ ]:
params = {
    'registry': '192.168.2.1:5001',
    'kafka_hosts': [f'{k}:{v}' for k, v in kafka_hosts.items()],
    'script_opts': [
        '-s', topic_src,
        '-d', topic_dst,
        '-b', ','.join(kafka_brokers),
        '-c', consumer_group_id,
        '--end',
    ],
}
if 'openpose_opts' in locals():
    params['script_opts'].extend(openpose_opts)

In [ ]:
from tempfile import TemporaryDirectory
from pathlib import Path
import json

!ansible {target} -m synchronize -a 'src=openpose/openpose-stream dest=.'
with TemporaryDirectory() as work_dir:
    vars_file = Path(work_dir) / 'vars.json'
    with vars_file.open(mode='w') as f:
        json.dump(params, f)
    !ansible {target} -m template -e @{vars_file} \
        -a 'src=openpose/openpose-stream/docker-compose.yml \
            dest=openpose-stream/'

## OpenPoseの実行

Kafka の画像ファイルをOpenPose で処理します。

まず、OpenPoseのコンテナイメージを取得します。

In [ ]:
!ansible {target} -a 'chdir=openpose-stream docker-compose pull'

OpenPoseの処理を行うプロセスを起動します。

In [ ]:
!ansible {target} -a 'chdir=openpose-stream docker-compose up -d'

状態を確認します。

In [ ]:
!ansible {target} -a 'chdir=openpose-stream docker-compose ps'

# 処理結果の確認

OpenPose で処理した画像をアニメーションで表示します。

## パラメータの指定

アニメーション表示用の kafka コンシューマのグループIDを指定します。

In [ ]:
animation_group_id = 'animation-001'

Kafka broker のアドレスを指定してください。

> ここではNotebook環境からみた場合のアドレスを指定してください。３章で指定した値と異なる場合があります。

In [ ]:
animation_broker_servers = [
    'broker-0:9092',
    'broker-1:9092',
    'broker-2:9092',
]

## 画像の表示

OpenPoseで処理した画像をアニメーションで表示します。

Kafkaから画像を取得するためのコンシューマをつくります。

> OpenPoseの処理対象となるトピックに

In [ ]:
from kafka import KafkaConsumer, TopicPartition

consumer = KafkaConsumer(
    bootstrap_servers=animation_broker_servers,
    group_id=animation_group_id
)
partitions = consumer.partitions_for_topic(topic_dst)
if partitions is not None:
    tps = [
        TopicPartition(topic_dst, x)
        for x in sorted(consumer.partitions_for_topic(topic_dst))
    ]
    consumer.assign(tps)

consumerの読み取り位置を最後に移動します。

In [ ]:
# 最後に処理した画像から表示します
if partitions is not None:
    for tp in tps:
        consumer.seek_to_end(tp)

アニメーション表示をの表示で画像を間引く場合は、次のセルで値を指定してください。

In [ ]:
image_step = 1

アニメーションを表示します。

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
from PIL import Image
from io import BytesIO
from itertools import islice

def update(msg):
    im = Image.open(BytesIO(msg.value))
    im_list = np.asarray(im)
    plt.clf()
    plt.imshow(im_list)
    
fig, ax = plt.subplots()
it = islice(consumer, 0, None, image_step) if 'image_step' in globals() else consumer
hoge = animation.FuncAnimation(fig, update, it, interval=25)
plt.show()

アニメーション表示が遅れているようでしたら、次のセルを実行してコンシューマのデータ取得位置を最後に移動してください。

In [ ]:
for tp in tps:
    consumer.seek_to_end(tp)

# 後始末

Consumerをクローズします。

In [ ]:
consumer.close()